# Preprocessing Aerial Detection Survey data

This script was used to preprocess the ADS geodatabase maintained by the USFS Forest Health Monitoring Program. The geodatabase contains all types of damage dating back to the 1940s for all of Oregon and Washington. We ran this script locally to clip all the individual year layers to the boundary of Malheur National Forest and then combined geodataframes and exported a GeoJSON containing all damage areas clipped to the MNF boundary for the years 1965-1990.

## Import packages

In [2]:
import geopandas as gpd
import os
import fiona
import pandas as pd
from glob import glob

## Define file paths and directories

In [2]:
# File paths
gdb_path = r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\downloaded_data\ads_gdb\AerialDetectionSurvey.gdb"  # Path to ADS GDB
ads_geojson_output_path = r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads"  # Output folder for clipped outbreak GeoJSONs
mnf_bounds = gpd.read_file(r"C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\extracted_mnf_subset\mnf_bounds.geojson")  # MNF boundary polygon

# Make sure output folder exists
os.makedirs(ads_geojson_output_path, exist_ok=True)

## Export yearly damage area layers overlapping MNF boundary

In [3]:
# Get all layers from GDB
layers = fiona.listlayers(gdb_path)

In [4]:
# Filter relevant DAMAGE_AREAS layers, excluding 'a_DAMAGE_AREAS_all'
damage_area_layers = [layer for layer in layers 
                      if "DAMAGE_AREAS" in layer and layer != "a_DAMAGE_AREAS_all"]

# Target CRS (optional - make sure all layers match for spatial ops)
target_crs = "EPSG:3857"
mnf_bounds = mnf_bounds.to_crs(target_crs)

for layer_name in damage_area_layers:
    try:
        print(f"Processing {layer_name}...")
        
        # Load the layer
        gdf = gpd.read_file(gdb_path, layer=layer_name)
        
        # Reproject if needed
        if gdf.crs != mnf_bounds.crs:
            gdf = gdf.to_crs(mnf_bounds.crs)
        
        # Clip to MNF boundary
        gdf_clipped = gpd.overlay(gdf, mnf_bounds, how='intersection')
        
        if not gdf_clipped.empty:
            out_path = os.path.join(ads_geojson_output_path, f"{layer_name}_mnf_overlap.geojson")
            gdf_clipped.to_file(out_path, driver="GeoJSON")
            print(f"Saved clipped layer to {out_path}")
        else:
            print(f"No overlap with MNF for {layer_name}. Skipped.")
            
    except Exception as e:
        print(f"Error processing {layer_name}: {e}")


Processing DAMAGE_AREAS_2021...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_2021_mnf_overlap.geojson
Processing DAMAGE_AREAS_2006...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_2006_mnf_overlap.geojson
Processing DAMAGE_AREAS_2008...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_2008_mnf_overlap.geojson
Processing DAMAGE_AREAS_2009...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_2009_mnf_overlap.geojson
Processing DAMAGE_AREAS_2004...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_2004_mnf_overlap.geojson
Processing DAMAGE_AREAS_2

C:\Users\imire\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_1988_mnf_overlap.geojson
Processing DAMAGE_AREAS_1989...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_1989_mnf_overlap.geojson
Processing DAMAGE_AREAS_1990...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_1990_mnf_overlap.geojson
Processing DAMAGE_AREAS_1991...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_1991_mnf_overlap.geojson
Processing DAMAGE_AREAS_1992...
Saved clipped layer to C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\DAMAGE_AREAS_1992_mnf_overlap.geojson
Processing DAMAGE_AREAS_1993...
Saved clipped layer to C:

## Combine and export WSB years 1965–1990 as one GeoJSON

In [5]:
print("Combining MNF-overlapping layers from 1965–1990...")

# Output file for combined data
combined_output_path = os.path.join(ads_geojson_output_path, "mnf_damage_1965_1990.geojson")

# Collect all the GeoJSONs
geojson_files = glob(os.path.join(ads_geojson_output_path, "*_mnf_overlap.geojson"))

# Read and concatenate only non-empty gdfs
gdfs = []
for fp in geojson_files:
    try:
        gdf = gpd.read_file(fp)
        if not gdf.empty and "SURVEY_YEAR":
            gdfs.append(gdf)
        else:
            print(f"Skipped (empty): {fp}")
    except Exception as e:
        print(f"Error reading {fp}: {e}")

# Combine gdfs and filter to 1965-1990 outbreaks
if gdfs:
    combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
    filtered = combined[(combined["SURVEY_YEAR"] >= 1965) & (combined["SURVEY_YEAR"] <= 1990)]

    if not filtered.empty:
        filtered.to_file(combined_output_path, driver="GeoJSON")
        print(f"Exported combined damage areas from 1965–1990 to: {combined_output_path}")
    else:
        print("No features found in the 1965–1990 range.")
else:
    print("No valid data to combine.")

Combining MNF-overlapping layers from 1965–1990...


C:\Users\imire\AppData\Local\Temp\ipykernel_7412\3133323853.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)


Exported combined damage areas from 1965–1990 to: C:\Users\imire\OneDrive - UW\Documents\GDA567\disturbance_interaction_analysis\script_outputs\mnf_ads\mnf_damage_1965_1990.geojson
